<a href="https://colab.research.google.com/github/ViBRUS/HPC-Codes-SPPU/blob/main/BECOMP67_HPC_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-78dfbsob
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-78dfbsob
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=579227ae08de6570a6e37f0267ba07d6e73eb56d3c1fd65a9429c78eae991f90
  Stored in directory: /tmp/pip-ephem-wheel-cache-w2yv7bti/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# HPC Assignment 1

### 1A.  Implement Parallel Reduction using Min, Max, Sum and Average operations.

In [ ]:
%%cu
#include <iostream>
#include <omp.h>
#include <time.h>
using namespace std;

int min(int a[],int n)
{
    int v = a[0];
    #pragma omp parallel for reduction(min:v)
    for (int i = 0; i < n; i++) 
    {
        if(a[i] < v)
            v = a[i];
    }
    return v;
}

int max(int a[],int n) 
{
    int v = a[0];
    #pragma omp parallel for reduction(max:v)
    for (int i =  0; i < n; i++) 
    {
        if(a[i] > v)
            v = a[i];
    }
    return v;
}

int sum(int a[],int n) {
  int sum = 0;
  #pragma omp parallel for reduction(+:sum)
  for (int i = 0; i < n; i++) {
    sum += a[i];
  }
  return sum;
}

float avg(int a[],int n) {
  return sum(a,n)/n;
}

int main() 
{
    clock_t t1,t2;
    int sz=100;
    int a[sz];
    
    srand(time(0));
    for(int i = 0;i<sz;i++)
    {
        a[i] = rand()%10000;
    }
    for(int i = 0;i<sz;i++)
    {
        cout<<a[i]<<"\t";
    }
    cout<<"\n";

    t1=clock();
    cout<<"Minimum Value: "<<min(a,sz)<<endl;
    t2=clock();
    printf("-Execution Time: %f",(double)(t2-t1)/CLOCKS_PER_SEC);

    t1=clock();
    cout<<"\nMaximum Value: "<<max(a,sz)<<endl;
    t2=clock();
    printf("-Execution Time: %f",(double)(t2-t1)/CLOCKS_PER_SEC);
 
    t1=clock();
    cout<<"\nSum: "<<sum(a,sz)<<"\n";
    t2=clock();
    printf("-Execution Time: %f",(double)(t2-t1)/CLOCKS_PER_SEC);

    t1=clock();
    cout<<"\nAverage: "<<avg(a,sz)<<"\n";
    t2=clock();
    printf("-Execution Time: %f",(double)(t2-t1)/CLOCKS_PER_SEC);
    return 0;
}

8544	5295	8564	8741	2018	7336	4789	1456	9817	4783	93	9416	4663	1493	2353	8473	6371	4155	8559	2773	6331	3470	9319	8715	1776	7886	7607	8126	9033	4379	8144	3929	6026	6708	9022	8045	396	3812	5853	213	8595	5946	9629	9610	3792	8334	8084	6515	2489	6643	9288	5172	6465	4959	3887	4593	2845	7846	2719	8230	8578	863	8512	4604	3923	7534	9001	4319	7698	4855	4533	6293	7153	514	2256	945	8849	340	7460	7690	3335	3100	2863	6152	8059	6750	745	7257	4597	9817	1839	9527	7032	6703	4131	7308	590	3133	1627	8288	
Minimum Value: 93
-Execution Time: 0.000015
Maximum Value: 9817
-Execution Time: 0.000003
Sum: 562896
-Execution Time: 0.000001
Average: 5628
-Execution Time: 0.000009


### 1B. Write CUDA programs that, given an N-element vector, find
1. The maximum element in the vector.
2. The minimum element in the vector.
3. The arithmetic mean of the vector.
4. The standard deviation of the values in the vector.
Test for input N and generate a randomized vector V of length N (N should be large). The program should generate output as the two computed maximum values as well as the time taken to find each value.


In [ ]:
%%cu
#include<iostream>
#include<cstdio>

using namespace std;

__global__ void maximum(int *a,int *b,int n)
{
	int block=256*blockIdx.x; 
	int max=0;

	for(int i=block;i<min(256+block,n);i++)
    {
	    if(max<a[i])
        {
		    max=a[i];	
		}
	}
	b[blockIdx.x]=max;
}

__global__ void minimum(int *a,int *b,int n)
{
	int block=256*blockIdx.x;
	int mini=7888888;

	for(int i=block;i<min(256+block,n);i++)
    {
	    if(mini>a[i])
        {
			mini=a[i];
		}
	}
	b[blockIdx.x]=mini;
}

void maxInVector()
{
	int n = 512;
	int a[n];
	cudaEvent_t start,end;

	for(int i=0;i<n;i++)
    {
		a[i]=i+1;
	}

	int *ad,*bd;
	int size=n*sizeof(int);
	
	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while(n>1)
    {
		maximum<<<grids,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time=0;
	cudaEventElapsedTime(&time,start,end);	
	int ans[2];
	cudaMemcpy(ans,ad,4,cudaMemcpyDeviceToHost);
	
	cout<<"The maximum element is "<<ans[0]<<endl;	
	cout<<"The time required do it is ";
	cout<<time<<endl;
}

void minInVector()
{
    int n = 512;
	int a[n];

	cudaEvent_t start,end;
	cudaEventCreate(&start);
	cudaEventCreate(&end);

	for(int i=0;i<n;i++)
    {
		a[i]=i+1;
	}
	
	int *ad,*bd;
	int size=n*sizeof(int);
	
	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

    int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);
	cudaEventRecord(start);
	while(n>1)
    {
		minimum<<<grids,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);
	float time=0;
	cudaEventElapsedTime(&time,start,end);

	int ans[2];
	cudaMemcpy(ans,ad,4,cudaMemcpyDeviceToHost);
	
	cout<<"The minimum element is "<<ans[0]<<endl;
	
	cout<<"The time required to do it is ";
	cout<<time<<endl;
}


__global__ void sum(int *a,int *b,int n){
	int block=256*blockIdx.x;
	int sum=0;

	for(int i=block;i<min(block+256,n);i++){
		sum=sum+a[i];
	}
	b[blockIdx.x]=sum;
}

__global__ void var(int *a,int *b,int n,float mean){
	int block=256*blockIdx.x;
	float sum=0;

	for(int i=block;i<min(block+256,n);i++){
	  sum=sum+(a[i]-mean)*(a[i]-mean);
	}
	b[blockIdx.x]=sum;
}


void meanInVector(){
  int n = 512;
	int a[n];
	for(int i=0;i<n;i++){
		a[i]=i+1;
	}

	int *ad,*bd;
	int size=n*sizeof(int);

	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);
	int p=n;

	cudaEvent_t start,end;
	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while(n>1){
		sum<<<grid,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

  float time=0;
	cudaEventElapsedTime(&time,start,end);
  cout<<"The time required is "<<time<<endl;

	int add[2];
	n=p;

	cudaMemcpy(add,ad,4,cudaMemcpyDeviceToHost);
	cout<<"The sum is  "<<add[0]<<endl;

	float mean=0.0f;
	mean=add[0]/(n*1.0f);
	cout<<"The mean is   "<<mean<<endl;
}

void sdInVector(){
  int n = 512;
	int a[n];

	for(int i=0;i<n;i++){
	  a[i]=i+1;
	}

	int *ad,*bd;
	int size=n*sizeof(int);

	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);

	int grids=ceil(n*1.0f/256.0f);
	cudaMalloc(&bd,grids*sizeof(int));

	dim3 grid(grids,1);
	dim3 block(1,1);

	int p=n;

	cudaEvent_t start,end;
	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while(n>1){
		sum<<<grid,block>>>(ad,bd,n);
		n=ceil(n*1.0f/256.0f);
		cudaMemcpy(ad,bd,n*sizeof(int),cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time=0;
	cudaEventElapsedTime(&time,start,end);
	cout<<"The time required is "<<time<<endl;

	int add[2];
	n=p;

	cudaMemcpy(add,ad,4,cudaMemcpyDeviceToHost);
	cout<<"The sum is  "<<add[0]<<endl;

	float mean=0.0f;
	mean=add[0]/(n*1.0f);

	cout<<"The mean is   "<<mean<<endl;
	cudaMalloc(&ad,size);
	cudaMemcpy(ad,a,size,cudaMemcpyHostToDevice);
	cudaMalloc(&bd,grids*sizeof(int));

	var<<<grid,block>>>(ad,bd,n,mean);
	n=ceil(n*1.0f/256.0f);	

	sum<<<grid,block>>>(bd,ad,n);

	cudaMemcpy(add,ad,4,cudaMemcpyDeviceToHost);

	float sd=sqrt(add[0]/p*1.0f);
	cout<<"The standard deviation is "<<sd<<endl;
}


int main()
{
    cout<<"The maximum element in the vector:-"<<endl;
    maxInVector();
    cout<<endl<<endl<<"The minimum element in the vector:-"<<endl;
    minInVector();
    cout<<endl<<endl<<"The arithmetic mean of the vector:-"<<endl;
    meanInVector();
    cout<<endl<<endl<<"The standard deviation of the values in the vector:-"<<endl;    
    sdInVector();
    return 0;
}

The maximum element in the vector:-
The maximum element is 512
The time required do it is 0.05824


The minimum element in the vector:-
The minimum element is 1
The time required to do it is 0.023296


The arithmetic mean of the vector:-
The time required is 0.031136
The sum is  131328
The mean is   256.5


The standard deviation of the values in the vector:-
The time required is 0.020128
The sum is  131328
The mean is   256.5
The standard deviation is 147.801

